In [3]:
# pip install geoaugment==0.1.1


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/30.1 MB ?

  DEPRECATION: Building 'antlr4-python3-runtime' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'antlr4-python3-runtime'. Discussion can be found at https://github.com/pypa/pip/issues/6334


   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ------------------ --------------------- 13.6/30.1 MB ? eta -:--:--
   ---

In [2]:
import geo_augment

print("GeoAugment version loaded successfully")

GeoAugment version loaded successfully


In [3]:
# # Import Required Modules
# import numpy as np
# import matplotlib.pyplot as plt

# from geo_augment.io.raster import RasterLoader
# from geo_augment.domains.floods.features import stack_flood_features
# # from geo_augment.domains.floods.api import (
# #     synthesize_flood_risk,
# #     synthesize_flood_labels
# # )
# from geo_augment.datasets.tiling import tile_raster
# from geo_augment.evaluation.floods import (
#     distribution_summary,
#     spatial_correlation
# )

import numpy as np
import matplotlib.pyplot as plt

# GeoAugment core
from geo_augment.io.raster import RasterLoader
from geo_augment.domains.floods.api import synthesize_flood_risk
from geo_augment.domains.floods.spec import (
    DEFAULT_FLOOD_SPEC,
    DEFAULT_FLOOD_CONSTRAINTS,
    DEFAULT_LATENT_SPEC,
)

# Evaluation utilities
from geo_augment.evaluation import (
    distribution_summary,
    spatial_correlation,
    flooded_area_ratio,
)

# Optional post-processing
from geo_augment.domains.floods.threshold import apply_percentile_threshold

# Dataset utilities
from geo_augment.datasets.tiling import tile_raster


ImportError: cannot import name 'synthesize_flood_risk' from 'geo_augment.domains.floods.api' (C:\Users\DELL\OneDrive\Documents\GeoAugmentProject\geo_augment\domains\floods\api.py)

In [ ]:
# Load Nigeria DEM
#dem_path = r"C:\path\to\nigeria_dem.tif"

dem = RasterLoader('nigeria.tif').read()

dem.shape, dem.min(), dem.max()

In [ ]:
# Visualize DEM
plt.figure(figsize=(6, 6))
plt.imshow(dem, cmap="terrain")
plt.colorbar(label="Elevation")
plt.title("Nigeria DEM")
plt.show()


In [ ]:
# Extract Flood-Relevant Features

features = stack_flood_features(dem)

features.shape


### GeoAugment converts raw DEMs into physically meaningful features.

These typically include:

Elevation

Slope

Flow accumulation

Base flood susceptibility

In [ ]:
# Inspect features

feature_names = [
    "Elevation",
    "Slope",
    "Flow Accumulation",
    "Base Flood Risk"
]

plt.figure(figsize=(12, 8))

for i, name in enumerate(feature_names):
    plt.subplot(2, 2, i + 1)
    plt.imshow(features[i], cmap="viridis")
    plt.title(name)
    plt.axis("off")

plt.tight_layout()
plt.show()


## Generate Continuous Flood Risk

GeoAugment generates **continuous flood risk values in [0, 1]**.

Interpretation:
- 0.0 → negligible flood susceptibility
- 1.0 → extreme flood susceptibility

This is **not a flood prediction**, but a **plausible risk surface**.


In [ ]:
# Generate Continuous Synthetic Flood Risk

synthetic_risk = synthesize_flood_risk(
    dem,
    n_samples=1
)[0]


### Instead of 0 / 1 flooded, we generate values in [0, 1] representing relative susceptibility.

In [ ]:
# Visualize synthetic flood risk

plt.figure(figsize=(6, 6))
plt.imshow(synthetic_risk, cmap="Blues")
plt.colorbar(label="Flood Risk (0–1)")
plt.title("Synthetic Flood Risk (Continuous)")
plt.show()


In [ ]:
# Convert Risk to Binary Flood Labels

binary_labels = synthesize_flood_labels(
    dem,
    n_samples=1,
    percentile=90.0
)[0]


In [ ]:
# Visualize binary flood labels

plt.figure(figsize=(6, 6))
plt.imshow(binary_labels, cmap="gray")
plt.title("Synthetic Flood Labels (Binary)")
plt.show()


In [ ]:
# Tile Dataset for ML / DL Training

X, y = tile_raster(
    features,
    binary_labels,
    tile_size=256,
    overlap=64
)

X.shape, y.shape


In [ ]:
# Visualize a sample tile

idx = 0

plt.figure(figsize=(10, 4))

plt.subplot(1, 2, 1)
plt.imshow(X[idx][0], cmap="terrain")
plt.title("Tile Elevation")

plt.subplot(1, 2, 2)
plt.imshow(y[idx], cmap="gray")
plt.title("Tile Flood Label")

plt.show()


In [ ]:
# Evaluate Synthetic Quality (Sanity Checks)

summary = distribution_summary(
    real=features[3],        # base flood risk
    synthetic=synthetic_risk
)

summary


In [ ]:
# Spatial correlation

corr = spatial_correlation(
    features[3],
    synthetic_risk
)

corr


Typical values:

> 0.6 → strong spatial agreement

< 0.3 → overly distorted (investigate)

In [ ]:
# Export Dataset (Optional)

from geo_augment.datasets.export import export_npz

export_npz(
    X,
    y,
    out_dir="geoaugment_output",
    name="nigeria_flood_tiles"
)


You can now train models using:

PyTorch

TensorFlow

scikit-learn